In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import ruptures as rpt  # our package
import pickle

from utils.driver_state import determine_drive_state_changed

## Driver change ensemble

In [9]:
# Create template dataframe for submission
with open("resources/annotations_public.pkl", "rb") as file:
    annotations = pickle.load(file)

submission_data = []
for video_name, video_annotations in annotations.items():
    for frame_number, frame_annotations in video_annotations.items():
        submission_data.append(
            {
                "ID": f"{video_name}_{frame_number}",
                "video": video_name,
                "frame": frame_number,
            }
        )

submission_df = pd.DataFrame(submission_data)


# Load predictions
driver_state_df = pd.read_csv("./submissions/results_driverstate_alltracks_bboxsizes.csv")
driver_state_df["frame"] = driver_state_df["ID"].apply(lambda x: int(x.split("_")[-1]))
driver_state_df["video"] = driver_state_df["ID"].apply(lambda x: "_".join(x.split("_")[:2]))

optical_flow_df = pd.read_csv("./submissions/results_driverstate_alltracks_opticalflow.csv")
optical_flow_df = optical_flow_df[["ID", "Driver_State_Changed"]]
optical_flow_df["frame"] = optical_flow_df["ID"].apply(lambda x: int(x.split("_")[-1]))
optical_flow_df["video"] = optical_flow_df["ID"].apply(lambda x: "_".join(x.split("_")[:2]))


# Create dict with index of first True value in Driver_State_Changed
driver_state_changes = defaultdict(list)
for video_name, video_group in driver_state_df.groupby(["video"]):
    state_changes = video_group.reset_index(drop=True)["Driver_State_Changed"]
    driver_state_changes[video_name].append(state_changes.idxmax())

for video_name, video_group in optical_flow_df.groupby(["video"]):
    state_changes = video_group.reset_index(drop=True)["Driver_State_Changed"]
    driver_state_changes[video_name].append(state_changes.idxmax())


# Calculate the mean index for True values
change_index = pd.DataFrame(driver_state_changes).T.mean(1).round().astype(int)
change_index_df = change_index.reset_index()
change_index_df.columns = ["video", "frame"]
change_index_df["change"] = True

# Merge the results with the main dataframe
final_df = pd.merge(
    submission_df[["ID", "video", "frame"]], change_index_df, how="left", on=["video", "frame"]
)
final_df["change"] = final_df["change"].fillna(False)
final_df["Driver_State_Changed"] = final_df.groupby("video")["change"].cumsum().astype(bool)

# Output final result to CSV
final_df = final_df[["ID", "Driver_State_Changed"]]
final_df.to_csv("./submissions/results_driverchange_ensemble.csv")

/tmp/ipykernel_60008/422262031.py:29: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, group in df1.groupby(['video']):
/tmp/ipykernel_60008/422262031.py:33: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, group in df2.groupby(['video']):
